Promising result in terms of generalization so far but it's a bit hard to train as it requires growing the network multiple times and adjusting the learning rate accordingly, rather than just growing one big model

In [ ]:
import numpy as np
import keras
import tensorflow as tf
from keras import layers
from keras import models
from keras.datasets import california_housing

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchinfo import summary
from torch.optim.lr_scheduler import StepLR

from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

import sys
import math

import numpy as np
import math

import numpy as np
import keras
import tensorflow as tf
from keras import layers
from keras import models
from keras.datasets import mnist

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchinfo import summary
from torch.optim.lr_scheduler import StepLR

import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

import numpy as np
import pandas as pd
from scipy.special import logit
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import LeaveOneOut, cross_val_predict, StratifiedKFold
from scipy.stats import norm

In [ ]:
class CustomLinearLayer(nn.Module):
    def __init__(self, input_size, output_size, init="looks_linear"):
        super(CustomLinearLayer, self).__init__()
        if init == "zero":
            self.linear = nn.Linear(input_size, output_size, bias=True)

            nn.init.zeros_(self.linear.bias)
            nn.init.zeros_(self.linear.weight)
        elif init == "looks_linear":
            if input_size * 2 != output_size:
                print("Output size must be twice that of input size")
                return
                
            self.linear = nn.Linear(input_size, output_size, bias=True)
            nn.init.zeros_(self.linear.bias)

            with torch.no_grad():
                weight = torch.zeros(input_size * 2, input_size)

                for i in range(self.linear.in_features):
                    weight[2 * i, i] = 1
                    weight[2 * i + 1, i] = -1

                self.linear.weight.copy_(weight)
                nn.init.zeros_(self.linear.bias)
                
            """ Example matrix: [
                [1, 0, 0],
                [-1, 0, 0],
                [0, 1, 0],
                [0, -1, 0],
                [0, 0, 1],
                [0, 0, -1]
            ] """

    def forward(self, x):
        return self.linear(x)

In [ ]:
class GradientBoostedResnet(nn.Module):        
    def __init__(self, input_size, output_size):
        super(GradientBoostedResnet, self).__init__()
        self.activation = nn.ReLU()
        
        self.first_layers = nn.ModuleList()
        self.last_layers = nn.ModuleList()

        self.starting_layer = CustomLinearLayer(input_size, output_size, init="zero")
        self.layer_size = input_size

    def grow_network(self):
        if len(self.first_layers) == 0:
            for param in self.starting_layer.parameters():
                param.requires_grad = False
        else:
            for param in self.first_layers[-1].parameters():
                param.requires_grad = False

            for param in self.last_layers[-1].parameters():
                param.requires_grad = False
                
        self.first_layers.append(CustomLinearLayer(self.layer_size, self.layer_size * 2, init="looks_linear").to(device))
        self.last_layers.append(CustomLinearLayer(self.layer_size * 2, self.output_size, init="zero").to(device))
        self.layer_size += self.layer_size * 2

    def forward(self, x):
        outputs = [x]
        for layer in self.first_layers:
            concatenated_outputs = torch.cat(outputs, dim=1)
            outputs.append(self.activation(layer(concatenated_outputs)))

        result = self.starting_layer(outputs[0])
        for i in range(len(self.last_layers)):
            result += self.last_layers[i](outputs[i + 1])

        return result